In [2]:
import pandas as pd

def split_series_title(series_title):
    # Split the series_title column into four columns based on its component parts delimited by commas
    parts = series_title.split(',')
    
    # Handle two or more parts in the title
    if len(parts) == 3:
        
        # Split first part into Series and Activity (delimited with a dash)
        first_part = parts[0].split('-')
        if len(first_part) == 2:
            series = first_part[0].strip()  # "Avg hrs per day"
            activity = first_part[1].strip()  # e.g., "Household activities (includes travel)"
        else:
            series = first_part[0].strip()  # If there is no dash, the whole part is the series
            activity = ""
        
        # Second part is Age group (e.g., "15-24 yrs")
        age_group = parts[1].strip()
        
        # Third part is Gender (e.g., "Women")
        gender = parts[2].strip()
        
    elif len(parts) == 2:  # If gender is missing, fill with MenAndWomen
        
        # Split first part into Series and Activity (delimited with a dash)
        first_part = parts[0].split('-')
        if len(first_part) == 2:
            series = first_part[0].strip()  
            activity = first_part[1].strip() 
        else:
            series = first_part[0].strip() 
            activity = ""
        
        # The second part is the Age group
        age_group = parts[1].strip()
        
        # No gender, so set to 'MenAndWomen'
        gender = "MenAndWomen"

    # Handle any unexpected cases
    else:
        series = activity = age_group = gender = None
    
    return pd.Series([series, activity, age_group, gender], index=['Series', 'Activity', 'Age group', 'Gender'])

def main():
    # File paths
    data_file = 'tu.data.1.AllData.txt'
    selected_series_file = 'tu_select_series.txt'
    output_csv_path = 'tu_processed_data.csv'

    # Load the selected series list; strip of whitespace
    with open(selected_series_file, 'r', encoding='utf-8') as f:
        selected_series = [line.strip() for line in f.readlines()]

    # Load the data file; strip whitespace from headers and from full seriesid column (necessary to faciitate merge below)
    data = pd.read_csv(data_file, sep='\t', engine='python')
    data.columns = data.columns.str.strip() 
    data['series_id'] = data['series_id'].str.strip()

    # Filter data to focus on selected seriesid's
    filtered_data = data[data['series_id'].isin(selected_series)]

    # Load the series file, which contains series titles; strip whitespace as above
    series_info = pd.read_csv('tu.series.txt', sep='\t', engine='python')
    series_info.columns = series_info.columns.str.strip()  
    series_info['series_id'] = series_info['series_id'].str.strip()

    # Filter series_info to focus on selected seriesid's
    filtered_series_info = series_info[series_info['series_id'].isin(selected_series)]

    # Merge the filtered data with the filered series_info to get the series titles
    merged_data = pd.merge(filtered_data, filtered_series_info[['series_id', 'series_title']], on='series_id', how='left')

    # Split the 'series_title' into the new columns and drop the 'series_title' column as it is no longer needed
    merged_data[['Series', 'Activity', 'Age group', 'Gender']] = merged_data['series_title'].apply(split_series_title)
    merged_data.drop(columns=['series_title'], inplace=True)

    # Save resulting data to CSV output file
    merged_data.to_csv(output_csv_path, index=False)

    print(f"Processed data saved to {output_csv_path}")

# Run the main function
if __name__ == "__main__":
    main()


Processed data saved to processed_data.csv
